In [58]:
import rasterio
import numpy as np
from rasterio.plot import show
import pylab 
from rasterio.windows import Window
from rasterio.mask import mask
import argparse
import os
import warnings
import torch
import torch.multiprocessing as mp
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
from data.datasets import *
import os
from glob import glob
import numpy as np
import torch
import pylab
import rasterio
import math
from rasterio.features import sieve
import numpy as np
import random
from model.models import *
from rasterio import Affine
from rasterio.merge import merge
from PIL import Image
from shapely.geometry.polygon import Polygon

In [59]:

image_dataset = rasterio.open('rgb/source1.tif', 'r')


ref_dataset = rasterio.open('ref.tif')
image_height = image_dataset.height
image_width = image_dataset.width
count = image_dataset.count
dtype = image_dataset.dtypes[0]


res_dataset = rasterio.open(
    'result.tif',
    'w+',
    driver='GTiff',
    height=image_height,
    width=image_width,
    count=3,
    dtype=dtype,
    crs=image_dataset.crs,
    transform=image_dataset.transform,
)
res_dataset.write(image_dataset.read([1,2,3]))


In [61]:
window_size = 256
# stride = window_size // 4
stride = 100


In [62]:

left, top = 0, 0
for y in range(0, image_height - window_size + 1, stride):
    is_located = False
    for x in range(0, image_width - window_size + 1, stride):
        window = image_dataset.read([1,2,3],window=rasterio.windows.Window(x, y, window_size, window_size))
        mask_window = (window[0] == 0) & (window[1] == 0) & (window[2] == 0)
        if np.sum(mask_window) < 2000:
            print("窗口左上角位置: ({}, {})".format(x, y))
            left, top = x, y
            is_located = True
            break
    if is_located:
        break
# left, top = 1408, 100

窗口左上角位置: (1400, 100)


In [63]:
window = image_dataset.read(window=rasterio.windows.Window(left, top, window_size, window_size))

In [66]:
def fix_window_pos(x, y):
    if x + window_size > image_width:
        x -= (x + window_size - image_width)
    elif x < 0:    
        x = 0
        
    if y + window_size > image_height:
        y -= (y + window_size - image_height)
    elif y < 0:
        y = 0    
    
    return x, y

In [67]:
gpu_ids = '2'

device = torch.device('cuda:{}'.format(gpu_ids))
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

image_size = [256, 256]
tfs = transforms.Compose([
                # transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5,0.5, 0.5])
        ])


generator = Generator().to(device)

model = 'saved_models/generator_2400.pth'
generator.load_state_dict(torch.load(model))
generator.eval()

def model(src, mask, ref):
    with torch.no_grad():
        img = tfs(src.transpose(1,2,0)).unsqueeze(0).to(device)
        ref_image = tfs(ref.transpose(1,2,0)).unsqueeze(0).to(device)
        mask = torch.from_numpy(mask).squeeze().unsqueeze(0).unsqueeze(0).bool().float().to(device)

        img_alpha = Variable(mask.type(Tensor)).to(device)
        mask_img = img * (1. - mask)
        cond_image = torch.concat([mask, ref_image, mask_img], dim=1)

        clip = Variable(cond_image.type(Tensor)).to(device)
        imgs_hr = Variable(img.type(Tensor)).to(device)

        gen = generator(clip)
        gen_f = gen * mask + img * (1 - mask)

        output = (gen_f + 1) * 0.5
        output = output.mul(255).add_(0.5).clamp_(0, 255).squeeze().to("cpu", torch.uint8).numpy().astype(np.uint8)
        
        return output

In [68]:
def get_window(x, y, window_size):
    window = res_dataset.read([1,2,3],window=rasterio.windows.Window(x, y, window_size, window_size))
    return window

def get_ori_window(x, y, window_size):
    window = image_dataset.read([1,2,3],window=rasterio.windows.Window(x, y, window_size, window_size))
    return window

In [69]:
import pyproj


def get_ref(x, y, window_size):
    # p_to = pyproj.CRS(4326)
    p_to = pyproj.CRS.from_user_input(ref_dataset.crs)
    p = pyproj.CRS.from_user_input(image_dataset.crs)
    transform_to_4326 = pyproj.Transformer.from_crs(p, p_to, always_xy=True)
    
    x1, y1 = x, y
    x2, y2 = x + window_size, y + window_size
    
    x_start, y_start = rasterio.transform.xy(image_dataset.transform, y1, x1)
    x_end, y_end = rasterio.transform.xy(image_dataset.transform, y2, x2)
    x_start, y_start = transform_to_4326.transform(x_start, y_start)
    x_end, y_end = transform_to_4326.transform(x_end, y_end)
    
    coords_4326 = [(x_start, y_start), (x_end, y_start), (x_end, y_end), (x_start, y_end)]
    
    
    copenhagen_poly = Polygon(coords_4326)
    cropped_ref, cropped_ref_transform = mask(ref_dataset, 
                                            shapes=[copenhagen_poly], 
                                            crop=True,
                                            all_touched=True)
    cropped_ref = cropped_ref[:3]
    cropped_ref = np.array(Image.fromarray(cropped_ref.transpose(1,2,0)).resize((window_size, window_size))).transpose(2,0,1)
    return cropped_ref

In [70]:
def bbox2mask(img_shape, bbox, dtype='uint8'):
    """Generate mask in ndarray from bbox.

    The returned mask has the shape of (h, w, 1). '1' indicates the
    hole and '0' indicates the valid regions.

    We prefer to use `uint8` as the data type of masks, which may be different
    from other codes in the community.

    Args:
        img_shape (tuple[int]): The size of the image.
        bbox (tuple[int]): Configuration tuple, (top, left, height, width)
        dtype (str): Indicate the data type of returned masks. Default: 'uint8'

    Return:
        numpy.ndarray: Mask in the shape of (h, w, 1).
    """

    height, width = img_shape[:2]

    mask = np.zeros((height, width, 1), dtype=dtype)
    mask[bbox[0]:bbox[0] + bbox[2], bbox[1]:bbox[1] + bbox[3], :] = 1

    return mask

def random_cropping_bbox(img_shape=(256,256), _type=0):
    h, w = img_shape

    if _type == 0: # 左
        top, left, height, width = 0, 0, h, w//2
    elif _type == 1: # 上
        top, left, height, width = 0, 0, h//2, w
    elif _type == 2: # 下
        top, left, height, width = h//2, 0, h//2, w
    elif _type == 3: # 右
        top, left, height, width = 0, w//2, h, w//2
        
    elif _type == 4: # 右下
        top, left, height, width = h//2, w//2, h//2, w//2
    elif _type == 5: # 右上
        top, left, height, width = 0, w//2, h//2, w//2
    elif _type == 6: # 左上
        top, left, height, width = 0, 0, h//2, w//2
    elif _type == 7: # 左下
        top, left, height, width = h//2, 0, h//2, w//2
        
    return (top, left, height, width)



In [74]:
def fill_window(x, y, window_size, mask_direction="right"):
    src = get_window(x, y, window_size)
    src_ori = get_ori_window(x, y, window_size)
    real_mask = (src_ori[0] == 0) & (src_ori[1] == 0) & (src_ori[2] == 0)
    if np.sum(real_mask) < 2000:
        return src

    if mask_direction == "right":
        _type = 3
    elif mask_direction == "left":
        _type = 0
    elif mask_direction == "top":
        _type = 1
    elif mask_direction == "bottom":
        _type = 2
    elif mask_direction == "right_bottom":
        _type = 4
    elif mask_direction == "right_top":
        _type = 5
    elif mask_direction == "left_top":
        _type = 6
    elif mask_direction == "left_bottom":
        _type = 7
    
    mask = bbox2mask((window_size, window_size), random_cropping_bbox((window_size, window_size), _type))
    ref = get_ref(x, y, window_size)
    filled = model(src, mask, ref)
    real_mask = np.expand_dims(real_mask, axis=0)
    mask = mask.transpose(2,0,1)
    filled = filled * mask + src * (1. - mask)
    filled = filled * real_mask + src_ori * (1. - real_mask)
    
    
    return filled
        

In [75]:
def fill_row(start_x, start_y, window_size):
    
    pos_x = start_x
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(pos_x, start_y)
        filled = fill_window(x, y, window_size)
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_x != x:
            break
        
        pos_x += stride
        
        cnt += 1
        
    pos_x = start_x
    while True:
    
        x, y = fix_window_pos(pos_x, start_y)
        filled = fill_window(x, y, window_size, mask_direction="left")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_x != x:
            break
        
        pos_x -= stride
        

In [76]:
def fill_col(start_x, start_y, window_size):
    
    pos_y = start_y
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="bottom")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y += stride
        
        cnt += 1
        
    pos_y = start_y
    
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="top")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y -= stride

In [77]:
def fill_col_right_bottom(start_x, start_y, window_size):
    
    pos_y = start_y
    pos_y += stride
    cnt = 0
    while True:
        x, y = fix_window_pos(start_x, pos_y)
        
        filled = fill_window(x, y, window_size, mask_direction="bottom")
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y = pos_y + stride
        x, y = fix_window_pos(start_x, pos_y)
        if pos_y != y:
            pos_y += stride
            continue
        
        x -= window_size // 2
        for _ in range((window_size // 2) // stride):
            filled = fill_window(x, pos_y, window_size, mask_direction="right")
            res_dataset.write(filled, window=Window(x, pos_y, window_size, window_size))
            x += stride
            
        x = x - (window_size // 2 - stride * ((window_size // 2) // stride))
        filled = fill_window(x, pos_y, window_size, mask_direction="right")
        res_dataset.write(filled, window=Window(x, pos_y, window_size, window_size))

        pos_y += stride
        
        cnt += 1

In [78]:
def fill_col_mask_right(start_x, start_y, window_size):
    
    pos_y = start_y
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="right_bottom")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y += stride
        
        cnt += 1
    
    pos_y = start_y
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="right_top")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y -= stride

In [79]:
def fill_col_mask_left(start_x, start_y, window_size):
    
    pos_y = start_y
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="left_bottom")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y += stride
        
        cnt += 1
        
    pos_y = start_y
    
    cnt = 0
    while True:
    
        x, y = fix_window_pos(start_x, pos_y)
        filled = fill_window(x, y, window_size, mask_direction="left_top")
        
        res_dataset.write(filled, window=Window(x, y, window_size, window_size))
        
        if pos_y != y:
            break
        
        pos_y -= stride

In [80]:
fill_row(left, top, window_size)
fill_col(left, top, window_size)

col_x = left + window_size // 2
row_y = top + window_size // 2
while True:
    
    x, _ = fix_window_pos(col_x, row_y)
    
    fill_col_mask_right(x, row_y, window_size)
    
    if x != col_x:
        break

    col_x += stride
    
    print(col_x)
    
col_x = left + window_size // 2
row_y = top + window_size // 2
while True:
    
    x, _ = fix_window_pos(col_x, row_y)
    
    fill_col_mask_left(x, row_y, window_size)
    
    if x != col_x:
        break

    col_x -= stride
    
    print(col_x)

    

1628
1728
1828
1928
2028
2128
2228
2328
2428
2528
2628
2728
2828
2928
3028
3128
3228
3328
3428
3528
3628
3728
3828
3928
4028
4128
4228
4328
4428
4528
4628
4728
4828
4928
5028
5128
5228
5328
5428
5528
5628
5728
5828
5928
6028
6128
6228
6328
6428
6528
6628
6728
6828
6928
7028
7128
7228
7328
7428
7528
7628
7728
7828
7928
8028
8128
8228
8328
8428
8528
8628
8728
1428
1328
1228
1128
1028
928
828
728
628
528
428
328
228
128
28
-72


In [83]:
res_dataset.close()